In [ ]:
%matplotlib inline
import gcsfs
import xarray as xr

# Get GCP map
fs = gcsfs.GCSFileSystem(project='sipn2-pangeo', token='anon', access='read_only')
gcsmap = gcsfs.mapping.GCSMap('sipn-data', gcs=fs, create=False)

# Open Zarr dataset
ds_gcs = xr.open_zarr(gcsmap)

ds_gcs

In [ ]:
ds_gcs.SIP.sel(model=b'MME').isel(init_end=0, fore_time=15).plot()